In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

 99% 497M/503M [00:14<00:00, 28.0MB/s]
100% 503M/503M [00:14<00:00, 35.2MB/s]


In [ ]:
!unzip *.zip && rm -r *.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/cnn_dailymail/train.csv")

In [ ]:
MAX_LEN = 512
SUMMARY_LEN = 150
TRAIN_SIZE = 5000

In [ ]:
train_df = train_df.iloc[0:TRAIN_SIZE, :].copy()
train_article_ls = list(train_df['article'])
train_highlight_ls = list(train_df['highlights'])

del train_df

In [ ]:
from random import sample

for index in sample(list(np.arange(len(train_article_ls))), 3):
  print("Original Text : ")
  print(train_article_ls[index])

  print("\n\nSummary Text : ")
  print(train_highlight_ls[index])
  
  print("==========================================================\n\n")

Original Text : 
She is a two-time Oscar-winning actress. But Dianne Wiest is reportedly struggling to find enough work to afford her New York City apartment. Speaking to the New York Times, the 66-year-old explained how she finds it difficult to earn enough film and television roles to pay the rent on her home, believed to be in Manhattan's West 79th Street. 'I have to move out of my apartment soon,' she admitted. A two-bed apartment at the actress's supposed apartment complex is currently advertised online at a rental price of $5,300 per month. Scroll down for video . Actress: Dianne Wiest (pictured) is reportedly struggling to find enough work to afford her New York home . Struggle: The 66-year-old explained how she finds it difficult to earn enough film and television roles to pay the rent at her home, believed to be in this apartment block in Manhattan's West 79th Street (pictured) Ms Wiest also revealed she never attempts to re-invent herself for a role. Instead, she leaves it to

In [ ]:
print(len(train_article_ls))
print(len(train_highlight_ls))

5000
5000


In [ ]:
df = pd.DataFrame(columns=['target_text', 'source_text'])
df['target_text'] = train_highlight_ls
df['source_text'] = ['summarize: '+ item for item in train_article_ls]

In [ ]:
df.head()

,target_text,source_text
0,"Bishop John Folda, of North Dakota, is taking ...",summarize: By . Associated Press . PUBLISHED: ...
1,Criminal complaint: Cop used his role to help ...,summarize: (CNN) -- Ralph Mata was an internal...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",summarize: A drunk driver who killed a young w...
3,Nina dos Santos says Europe must be ready to a...,summarize: (CNN) -- With a breezy sweep of his...
4,Fleetwood top of League One after 2-0 win at S...,summarize: Fleetwood are the only team still t...


In [ ]:
!pip install simplet5 -q

     |████████████████████████████████| 1.3 MB 39.2 MB/s 
     |████████████████████████████████| 3.5 MB 49.1 MB/s 
     |████████████████████████████████| 527 kB 74.0 MB/s 
     |████████████████████████████████| 952 kB 66.5 MB/s 
     |████████████████████████████████| 419 kB 75.0 MB/s 
     |████████████████████████████████| 141 kB 77.0 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
     |████████████████████████████████| 829 kB 64.0 MB/s 
     |████████████████████████████████| 880 kB 63.1 MB/s 
     |████████████████████████████████| 6.6 MB 55.3 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
MAX_EPOCHS = 5

In [ ]:
model.train(train_df=df[0:(int)(0.7*TRAIN_SIZE)],
            eval_df=df[(int)(0.7*TRAIN_SIZE):TRAIN_SIZE],
            source_max_token_len=MAX_LEN,
            target_max_token_len=SUMMARY_LEN,
            batch_size=8, max_epochs=MAX_EPOCHS, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
import os
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

outputs/simplet5-epoch-4-train-loss-1.1003-val-loss-1.535


In [ ]:
model.load_model("t5","./"+model_path, use_gpu=True)

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [ ]:
from random import sample

for index in sample(list(np.arange(len(train_article_ls))), 5):
  print("Original Text : ")
  print(train_article_ls[index])
  print("\n\nSummaryText : ")
  print(train_highlight_ls[index])
  print("\n\n Fine tuned Text : ")
  print(model.predict(train_article_ls[index]))

Token indices sequence length is longer than the specified maximum sequence length for this model (1213 > 512). Running this sequence through the model will result in indexing errors


Original Text : 
A bureau de change boss who recruited his wife and son to help clean up nearly £170 million-worth of criminal cash has been jailed for nine years. Ussama El-Kurd, 64, known as 'Sammy the Kurd' in underworld circles, was on bail pending an appeal for a previous money laundering conviction when he set up the business in 2006. He enlisted his wife Randa, 51, and son Husain, 22, to help run the Kensington Money Exchange in upmarket Notting Hill, west London. Ussama El-Kurd, known as 'Sammy the Kurd', was on bail for a previous money laundering conviction when he set up the business in 2006 . Their efforts allowed the proceeds of drug dealing, violent robberies and other major crimes to be spent freely by offenders. The Palestinian-born family ran their 'bespoke money laundering service' in Kensington Church Street. No records of the dodgy transactions were kept, but authorities say the business must have helped to wash at least £169m until it was shut down in 2010. They es

In [ ]:
text = "Prolific tweeter Elon Musk said he is buying British football club Manchester United in a thread joking about his political allegiances. The Tesla chief executive officer and world’s richest person made the claim in a tweet thread where he stated he supported the left half of the Republican Party and the right half of the Democrats. Musk has a track record of using Twitter to make jokes and troll the media, who follow what he tweets closely. He has also run afoul of the US Securities and Exchange Commission, most infamously for tweets in 2018 about taking Tesla private. That led to securities fraud charges, and a settlement that included appointing a “Twitter sitter” to approve Musk’s Tesla-related tweets."
summary = model.predict(text)
print(" ".join(summary))

['Tesla chief Elon Musk said he is buying Manchester United in a tweet thread joking about his political allegiances. The claim was made by the Tesla CEO and world’s richest person. Musk has a track record of using Twitter to make jokes and troll the media, most infamous for tweets in 2018 about taking Tesla private.']
